Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto : Question 3

In [2]:
# import first all required dependencies for the project
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [4]:
toronto_data = pd.read_csv (r'torontomerged.csv')
toronto_data = toronto_data.drop('Unnamed: 0', axis=1)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Harbourfront, Regen...",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei...",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [6]:
place = 'Toronto, ON'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(place)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207


In [12]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_Toronto

In [10]:
# I will focus on Toronto boroughs
print (np.unique(toronto_data.Borough))

['Central Toronto' 'Downtown Toronto' 'East Toronto' 'East York'
 'Etobicoke' 'Mississauga' 'North York' "Queen's Park" 'Scarborough'
 'West Toronto' 'York']


In [25]:
# define foursquare necessities
CLIENT_ID = 'WT3ZB3MYCPKTZJJPN1DFDMVNHKFC4XU0C5GH4QN45KS443LD' # your Foursquare ID
CLIENT_SECRET = 'WNSYU3DXBLTT12SEOQDIWGV2ELCYEKV3ALTAGD5XUBEO5UPS' # your Foursquare Secret
VERSION = '20191606' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# define API request URL

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: WT3ZB3MYCPKTZJJPN1DFDMVNHKFC4XU0C5GH4QN45KS443LD
CLIENT_SECRET:WNSYU3DXBLTT12SEOQDIWGV2ELCYEKV3ALTAGD5XUBEO5UPS


first ı will break down toronto borughs and define central toronto to explore this map in foursquare


In [27]:

#central toronto creation
central_toronto = pd.DataFrame()
central_toronto = toronto_data[toronto_data['Borough'] == 'Central Toronto'].reset_index(drop=True)


central_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West, Forest Hi...",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [28]:
central_toronto_venues = getNearbyVenues(names=central_toronto['Neighborhood'],
                                   latitudes=central_toronto['Latitude'],
                                   longitudes=central_toronto['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West
North Toronto West
The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East, Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West


In [30]:
central_toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.72802,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.72802,-79.38879,Granite Club,43.733043,-79.381986,Gym / Fitness Center
2,Lawrence Park,43.72802,-79.38879,Tim Hortons,43.727324,-79.379563,Coffee Shop
3,Lawrence Park,43.72802,-79.38879,Glendon Bookstore,43.727024,-79.378976,Bookstore
4,Lawrence Park,43.72802,-79.38879,Glendon Forest,43.727226,-79.378413,Trail


In [31]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,100,100,100,100,100,100
Davisville North,100,100,100,100,100,100
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",80,80,80,80,80,80
"Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West",42,42,42,42,42,42
Lawrence Park,8,8,8,8,8,8
"Moore Park, Summerhill East, Moore Park, Summerhill East",64,64,64,64,64,64
North Toronto West,44,44,44,44,44,44
Roselawn,25,25,25,25,25,25
"The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville, The Annex, North Midtown, Yorkville",100,100,100,100,100,100


In [32]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 129 uniques categories.


In [34]:
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Yoga Studio,American Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Baseball Field,Beer Bar,Bike Shop,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Clothing Store,Coffee Shop,College Gym,College Quad,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,Office,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [35]:
central_toronto_grouped = central_toronto_onehot.groupby('Neighborhood').mean().reset_index()
central_toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Baseball Field,Beer Bar,Bike Shop,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Chinese Restaurant,Chiropractor,Clothing Store,Coffee Shop,College Gym,College Quad,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Office,Park,Pastry Shop,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Poutine Place,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Davisville,0.010000,0.000000,0.00,0.00,0.00,0.00000,0.010000,0.010000,0.000000,0.00,0.00,0.0000,0.020000,0.010000,0.01,0.01,0.00,0.000000,0.00,0.040000,0.000000,0.000000,0.01,0.00,0.010000,0.000000,0.000000,0.0100,0.090000,0.000,0.000,0.00,0.01,0.00,0.00,0.010000,0.00,0.03,0.010000,0.00,0.00,0.000000,0.01,0.030000,0.000000,0.000000,0.0100,0.000000,0.010000,0.01,0.000000,0.020000,0.01,0.000000,0.0000,0.01,0.01,0.000000,0.030000,0.00000,0.000000,0.00,0.00,0.00,0.0100,0.02,0.03,0.00,0.01,0.070000,0.010000,0.000000,0.00,0.00000,0.00,0.00,0.000,0.00000,0.020000,0.02000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.00,0.00000,0.00,0.020000,0.030000,0.00000,0.01,0.00,0.00,0.030000,0.010000,0.000000,0.030000,0.01,0.000000,0.02000,0.00,0.01,0.000000,0.01,0.000000,0.000000,0.0000,0.01,0.0100,0.060000,0.01,0.000000,0.000000,0.010000,0.010000,0.00,0.01,0.000000,0.000000,0.01000,0.00,0.010000,0.010000,0.01
1,Davisville North,0.020000,0.000000,0.00,0.00,0.00,0.00000,0.010000,0.000000,0.000000,0.00,0.00,0.0000,0.020000,0.010000,0.01,0.01,0.01,0.010000,0.00,0.040000,0.010000,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.0000,0.130000,0.000,0.000,0.00,0.00,0.00,0.00,0.010000,0.00,0.03,0.020000,0.01,0.00,0.010000,0.00,0.050000,0.000000,0.020000,0.0000,0.000000,0.010000,0.01,0.000000,0.010000,0.00,0.000000,0.0000,0.00,0.01,0.020000,0.030000,0.00000,0.000000,0.00,0.00,0.00,0.0100,0.01,0.01,0.00,0.00,0.050000,0.010000,0.010000,0.00,0.00000,0.00,0.00,0.010,0.00000,0.020000,0.00000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.000000,0.010000,0.00,0.00000,0.00,0.030000,0.040000,0.00000,0.01,0.00,0.00,0.010000,0.020000,0.000000,0.020000,0.01,0.010000,0.01000,0.00,0.01,0.000000,0.01,0.010000,0.020000,0.0000,0.00,0.0100,0.030000,0.01,0.000000,0.010000,0.000000,0.000000,0.00,0.01,0.010000,0.000000,0.01000,0.00,0.010000,0.010000,0.00
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.012500,0.012500,0.00,0.00,0.00,0.02500,0.012500,0.012500,0.000000,0.00,0.00,0.0125,0.012500,0.012500,0.00,0.00,0.00,0.012500,0.00,0.025000,0.000000,0.012500,0.00,0.00,0.000000,0.000000,0.012500,0.0125,0.112500,0.000,0.000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00

In [48]:
#No we will generate a data frame for the 10 top venues for each neighborhood.

num_top_venues = 5

for hood in central_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_toronto_grouped[central_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2    Sushi Restaurant  0.06
3                Café  0.04
4   Indian Restaurant  0.03


----Davisville North----
                  venue  freq
0           Coffee Shop  0.13
1    Italian Restaurant  0.05
2  Fast Food Restaurant  0.05
3           Pizza Place  0.04
4                  Café  0.04


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West, Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                venue  freq
0         Coffee Shop  0.11
1                Park  0.06
2  Italian Restaurant  0.05
3         Pizza Place  0.05
4    Sushi Restaurant  0.05


----Forest Hill North, Forest Hill West, Forest Hill North, Forest Hill West----
           

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
central_venues_sorted = pd.DataFrame(columns=columns)
central_venues_sorted['Neighborhood'] = central_toronto_grouped['Neighborhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    central_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

central_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Restaurant,Fast Food Restaurant,Gym,Pub,Pizza Place,Indian Restaurant
1,Davisville North,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pizza Place,Café,Dessert Shop,Sushi Restaurant,Gym,Pharmacy,Yoga Studio
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Pizza Place,Pub,Bagel Shop,Thai Restaurant,Gym,Gym / Fitness Center
3,"Forest Hill North, Forest Hill West, Forest Hi...",Park,Italian Restaurant,Café,Coffee Shop,Burger Joint,Japanese Restaurant,Trail,Sushi Restaurant,Bank,Sandwich Place
4,Lawrence Park,College Quad,College Gym,Café,Bookstore,Coffee Shop,Park,Trail,Gym / Fitness Center,Design Studio,Dessert Shop


In [42]:
# set number of clusters
kclusters = 5

central_toronto_grouped_clustering = central_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 1, 0, 4, 2, 0], dtype=int32)

In [46]:
# add clustering labels
central_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_central_merged = central_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_central_merged = toronto_central_merged.join(central_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_central_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,College Quad,College Gym,Café,Bookstore,Coffee Shop,Park,Trail,Gym / Fitness Center,Design Studio,Dessert Shop
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Italian Restaurant,Sushi Restaurant,Café,Asian Restaurant,Coffee Shop,Bank,Pharmacy,Skating Rink,Deli / Bodega,Clothing Store
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pizza Place,Café,Dessert Shop,Sushi Restaurant,Gym,Pharmacy,Yoga Studio
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West, Forest Hi...",43.696948,-79.411307,3,Park,Italian Restaurant,Café,Coffee Shop,Burger Joint,Japanese Restaurant,Trail,Sushi Restaurant,Bank,Sandwich Place
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,4,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Diner,Café,Park,Mexican Restaurant,Deli / Bodega,Restaurant,Rental Car Location


In [47]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
toronto_central_merged['Cluster Labels'] =toronto_central_merged['Cluster Labels'].astype(int)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_central_merged['Latitude'], toronto_central_merged['Longitude'], toronto_central_merged['Neighborhood'], toronto_central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters